<a href="https://colab.research.google.com/github/vineelbhatti/NbaTeamFit/blob/main/NbaTeamFit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder

# Load the college basketball player stats dataset
nba_stats = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/2022-23_NBA_Player_Stats.csv', encoding = 'latin', sep=';')
height_weight = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/HeightAndWeight.csv')
college_stats = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/CollegeBasketball.csv')
height_weight.rename(columns={'Name':'Player'}, inplace=True)
nba_stats = pd.merge(nba_stats, height_weight, on='Player')
nba_stats = nba_stats.drop(columns=['Age_y', 'Height', 'Points', 'Rebounds', 'Assists', 'Team', 'Position'])
nba_stats['feet'], nba_stats['inches'] = nba_stats['Height_i'].astype(str).str.split('.', 1).str
nba_stats['inches'] = nba_stats['inches'].astype(int) + nba_stats['feet'].astype(int) * 12
nba_stats = nba_stats.drop('feet', axis=1)


nba_stats.loc[nba_stats['Player']=='Steven Adams']




<ipython-input-54-e5ad989c6860>:12: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  nba_stats['feet'], nba_stats['inches'] = nba_stats['Height_i'].astype(str).str.split('.', 1).str


,Rk,Player,Pos,Age_x,Tm,G,GS,MP,FG,FGA,...,STL,BLK,TOV,PF,PTS,Height_i,Weight,College,Salary,inches
1,2,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,...,0.9,1.1,1.9,2.3,8.6,6.11,265,Pittsburgh,17073171.0,83


In [56]:
# Load a list of NBA team names
nba_teams = ['ATL', 'BOS', 'BRK', 'CHA', 'CHO', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 'OKC', 'ORL', 'PHI', 'PHX', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS']

nba_stats = nba_stats.rename({'FG%': 'FG'})

# Define the feature columns
#feature_cols = ['PTS', 'ORB', 'DRB', 'AST', 'FG%', 'BLK', 'STL', 'Height_i', 'Weight']
feature_cols = ['PTS', 'ORB', 'DRB', 'AST', 'FG', 'BLK', 'STL']


# Prompt the user to enter their college stats
# PTS = float(input("Enter the average points per game: "))
# ORB = float(input("Enter the average offensive rebounds per game: "))
# DRB = float(input("Enter the average defensive rebounds per game: "))
# AST = float(input("Enter the average assists per game: "))
# FG = float(input("Enter the field goal percentage: "))
# BLK = float(input("Enter the avaerage blocks per game: "))
# STL = float(input("Enter the average steals per game: "))
# Height_i = float(input("Enter the height in decimal format: "))
# Weight = float(input("Enter the weight in pounds: "))

player = input('Enter prospect name: ')
selected_row = college_stats.loc[college_stats['player_name'] == player]
PTS = float(selected_row['pts'])
ORB = float(selected_row['oreb'])
DRB = float(selected_row['dreb'])
AST = float(selected_row['ast'])
FG = float(selected_row['TS_per'])
BLK = float(selected_row['blk'])
STL = float(selected_row['stl'])
archetype = str(selected_row['Unnamed: 64'])

wing_stats = nba_stats
wing_stats = wing_stats[wing_stats['inches'] >= 77]
wing_stats = wing_stats[wing_stats['inches'] <= 82]
wing_stats.head(5)

if 'Wing' in archetype:
  # Split the data into training and testing sets
  print('F')
  X_train = wing_stats[feature_cols]
  y_train = wing_stats['Tm']


# Create the KNN model
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Use the KNN model to predict the NBA team fit
# player_stats = [[PTS, ORB, DRB, AST, FG, BLK, STL, Height_i, Weight]]
player_stats = [[PTS, ORB, DRB, AST, FG, BLK, STL]]
distances, indices = knn.kneighbors(player_stats)
team_name = knn.predict(player_stats)[0]
team_index = nba_teams.index(team_name)
nba_team = nba_teams[team_index]

# Print the predicted NBA team fit
print("Based on your college stats, the best NBA team fit for you is:", nba_team)

print("Here are some similar players in the NBA:")
for index in indices[0]:
    print(nba_stats.loc[index, 'Player'])



Enter prospect name: Drew Timme
Based on your college stats, the best NBA team fit for you is: BOS
Here are some similar players in the NBA:
Joel Embiid
Bradley Beal
Maxi Kleber
Josh Christopher
Dalano Banton


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  warnings.warn(
